#### Importing Libraries

In [197]:
#import the libraries
import pandas as pd
import numpy as np
#import the plotting libraries
import warnings
warnings.filterwarnings("ignore")
import piplite
await piplite.install('seaborn')
await piplite.install('matplotlib')
#import the plotting libraries
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

<class 'OSError'>: Not available

#### Read CSV file

In [198]:
pd.options.display.float_format = '{:.0f}'.format
df=pd.read_csv("loan.csv")
df.head()


,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
0,1077501,1296599,5000,5000,4975,36 months,10.65%,163,B,B2,...,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN
1,1077430,1314167,2500,2500,2500,60 months,15.27%,60,C,C4,...,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN
2,1077175,1313524,2400,2400,2400,36 months,15.96%,84,C,C5,...,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN
3,1076863,1277178,10000,10000,10000,36 months,13.49%,339,C,C1,...,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN
4,1075358,1311748,3000,3000,3000,60 months,12.69%,68,B,B5,...,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN


In [199]:
df.shape


(39717, 111)

### Data Handling and Cleaning

In [200]:
df.isnull().sum(axis=0)

id                                0
member_id                         0
loan_amnt                         0
funded_amnt                       0
funded_amnt_inv                   0
                              ...  
tax_liens                        39
tot_hi_cred_lim               39717
total_bal_ex_mort             39717
total_bc_limit                39717
total_il_high_credit_limit    39717
Length: 111, dtype: int64

In [201]:
# drop columns with all NaN's
df = df.dropna(axis=1, how='all')
df.shape

(39717, 57)

In [202]:
df.isnull().sum()

id                                0
member_id                         0
loan_amnt                         0
funded_amnt                       0
funded_amnt_inv                   0
term                              0
int_rate                          0
installment                       0
grade                             0
sub_grade                         0
emp_title                      2459
emp_length                     1075
home_ownership                    0
annual_inc                        0
verification_status               0
issue_d                           0
loan_status                       0
pymnt_plan                        0
url                               0
desc                          12940
purpose                           0
title                            11
zip_code                          0
addr_state                        0
dti                               0
delinq_2yrs                       0
earliest_cr_line                  0
inq_last_6mths              

In [203]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39717 entries, 0 to 39716
Data columns (total 57 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   id                          39717 non-null  int64  
 1   member_id                   39717 non-null  int64  
 2   loan_amnt                   39717 non-null  int64  
 3   funded_amnt                 39717 non-null  int64  
 4   funded_amnt_inv             39717 non-null  float64
 5   term                        39717 non-null  object 
 6   int_rate                    39717 non-null  object 
 7   installment                 39717 non-null  float64
 8   grade                       39717 non-null  object 
 9   sub_grade                   39717 non-null  object 
 10  emp_title                   37258 non-null  object 
 11  emp_length                  38642 non-null  object 
 12  home_ownership              39717 non-null  object 
 13  annual_inc                  397

In [204]:
null_cols = df.columns[df.isnull().any()]
null_df = df[null_cols].isnull().sum().to_frame(name='Null Count')\
          .merge(df[null_cols].isnull().mean().mul(100).to_frame(name='Null Percent'), left_index=True, right_index=True)
null_df_sorted = null_df.sort_values(by='Null Count', ascending=False)
print(null_df_sorted)

                            Null Count  Null Percent
next_pymnt_d                     38577            97
mths_since_last_record           36931            93
mths_since_last_delinq           25682            65
desc                             12940            33
emp_title                         2459             6
emp_length                        1075             3
pub_rec_bankruptcies               697             2
last_pymnt_d                        71             0
collections_12_mths_ex_med          56             0
chargeoff_within_12_mths            56             0
revol_util                          50             0
tax_liens                           39             0
title                               11             0
last_credit_pull_d                   2             0


In [205]:
df=df.drop(['next_pymnt_d','mths_since_last_record','mths_since_last_delinq'],axis=1)
df.shape

(39717, 54)

#### Missing Value Treatment

In [206]:
df[['desc','emp_title','title']]=df[['desc','emp_title','title']].fillna('Others')

In [207]:
null_cols = df.columns[df.isnull().any()]
null_df = df[null_cols].isnull().sum().to_frame(name='Null Count')\
          .merge(df[null_cols].isnull().mean().mul(100).to_frame(name='Null Percent'), left_index=True, right_index=True)
null_df_sorted = null_df.sort_values(by='Null Count', ascending=False)
print(null_df_sorted)

                            Null Count  Null Percent
emp_length                        1075             3
pub_rec_bankruptcies               697             2
last_pymnt_d                        71             0
collections_12_mths_ex_med          56             0
chargeoff_within_12_mths            56             0
revol_util                          50             0
tax_liens                           39             0
last_credit_pull_d                   2             0


In [208]:
df['emp_title'].value_counts()

Others                             2459
US Army                             134
Bank of America                     109
IBM                                  66
AT&T                                 59
                                   ... 
AMEC                                  1
lee county sheriff                    1
Bacon County Board of Education       1
Hotpads                               1
Evergreen Center                      1
Name: emp_title, Length: 28821, dtype: int64

In [209]:
df=df.drop(['emp_title'],axis=1)
df.shape

(39717, 53)

In [210]:
df['term'] = df['term'].str.replace('months', '')
df['term'] = df['term'].astype(int)

In [211]:
df['int_rate'] = df['int_rate'].str.replace('%', '')
df['int_rate'] = df['int_rate'].astype(float)

In [212]:
df['emp_length'] = df['emp_length'].fillna('-1')
df['emp_length'] = df['emp_length'].str.replace('years', '')
df['emp_length'] = df['emp_length'].str.replace('year', '')
df['emp_length'] = df['emp_length'].str.replace('+', '')    
df['emp_length'] = df['emp_length'].str.replace('< 1', '0') 
df['emp_length'] = df['emp_length'].astype(int)

In [213]:
df['annual_inc'].describe()

count     39717
mean      68969
std       63794
min        4000
25%       40404
50%       59000
75%       82300
max     6000000
Name: annual_inc, dtype: float64

In [214]:
df=df.drop(['collections_12_mths_ex_med','policy_code','acc_now_delinq','chargeoff_within_12_mths','delinq_amnt','pub_rec_bankruptcies','tax_liens'],axis=1)
df.shape

(39717, 46)

In [215]:
df['term'].value_counts()

36    29096
60    10621
Name: term, dtype: int64

In [216]:
df.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,last_credit_pull_d,application_type
0,1077501,1296599,5000,5000,4975,36,11,163,B,B2,...,5834,5000,863,0,0,0,Jan-15,172,May-16,INDIVIDUAL
1,1077430,1314167,2500,2500,2500,60,15,60,C,C4,...,1009,456,435,0,117,1,Apr-13,120,Sep-13,INDIVIDUAL
2,1077175,1313524,2400,2400,2400,36,16,84,C,C5,...,3006,2400,606,0,0,0,Jun-14,650,May-16,INDIVIDUAL
3,1076863,1277178,10000,10000,10000,36,13,339,C,C1,...,12232,10000,2215,17,0,0,Jan-15,357,Apr-16,INDIVIDUAL
4,1075358,1311748,3000,3000,3000,60,13,68,B,B5,...,3513,2476,1037,0,0,0,May-16,68,May-16,INDIVIDUAL


In [217]:
df.grade.value_counts()

B    12020
A    10085
C     8098
D     5307
E     2842
F     1049
G      316
Name: grade, dtype: int64

In [218]:
df['verification_status'].value_counts()

Not Verified       16921
Verified           12809
Source Verified     9987
Name: verification_status, dtype: int64

In [219]:
df['loan_status'].value_counts()

Fully Paid     32950
Charged Off     5627
Current         1140
Name: loan_status, dtype: int64

In [225]:
res = pd.pivot_table(data=df,index="loan_status",columns="verification_status",values="member_id",aggfunc='count')
print(res)

verification_status  Not Verified  Source Verified  Verified
loan_status                                                 
Charged Off                  2142             1434      2051
Current                       227              310       603
Fully Paid                  14552             8243     10155


In [ ]:
39717

In [239]:
df[df['loan_status']=='Fully Paid'].count()

id                         32950
member_id                  32950
loan_amnt                  32950
funded_amnt                32950
funded_amnt_inv            32950
term                       32950
int_rate                   32950
installment                32950
grade                      32950
sub_grade                  32950
emp_length                 32950
home_ownership             32950
annual_inc                 32950
verification_status        32950
issue_d                    32950
loan_status                32950
pymnt_plan                 32950
url                        32950
desc                       32950
purpose                    32950
title                      32950
zip_code                   32950
addr_state                 32950
dti                        32950
delinq_2yrs                32950
earliest_cr_line           32950
inq_last_6mths             32950
open_acc                   32950
pub_rec                    32950
revol_bal                  32950
revol_util

In [241]:
df['int_rate'].describe()

count   39717
mean       12
std         4
min         5
25%         9
50%        12
75%        15
max        25
Name: int_rate, dtype: float64

In [244]:
res = pd.pivot_table(data=df,index="loan_status",columns="int_rate",values="member_id",aggfunc='count')
print(res)

int_rate     5   6   6   6   6   6   6   7   7   7   ...  23  23  23  23  24  \
loan_status                                          ...                       
Charged Off  20  16  19 NaN   5  10   1  27  29   5  ...   4   1   5   1   4   
Current     NaN NaN NaN   2 NaN NaN NaN NaN NaN NaN  ... NaN   1   1 NaN   3   
Fully Paid  553 394 328  16 442 242  57 278 367 163  ...   4 NaN   3   8   2   

int_rate     24  24  24  24  25  
loan_status                      
Charged Off   1   2   1   1 NaN  
Current     NaN   2 NaN NaN   1  
Fully Paid    3   7   2 NaN NaN  

[3 rows x 371 columns]


In [246]:
df['int_rate_Bucket'] = pd.qcut(df['int_rate'], [0, 0.2, 0.4, 0.6, 0.8, 1], ["5%","6%-9%","10%-12%","13%-15%","16%-25%"])

0          6%-9%
1        13%-15%
2        16%-25%
3        13%-15%
4        10%-12%
          ...   
39712      6%-9%
39713      6%-9%
39714      6%-9%
39715         5%
39716    13%-15%
Name: int_rate_Bucket, Length: 39717, dtype: category
Categories (5, object): ['5%' < '6%-9%' < '10%-12%' < '13%-15%' < '16%-25%']